In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
import numpy as np
import pandas as pd

traffic_df = pd.read_csv('/content/gdrive/MyDrive/CIC-Darknet/Darknet.csv')
traffic_df.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Label.1
0,10.152.152.11-216.58.220.99-57158-443-6,10.152.152.11,57158,216.58.220.99,443,6,24/07/2015 04:09:48 PM,229,1,1,...,0,0,0,0,0.000000e+00,0.000,0.000000e+00,0.000000e+00,Non-Tor,AUDIO-STREAMING
1,10.152.152.11-216.58.220.99-57159-443-6,10.152.152.11,57159,216.58.220.99,443,6,24/07/2015 04:09:48 PM,407,1,1,...,0,0,0,0,0.000000e+00,0.000,0.000000e+00,0.000000e+00,Non-Tor,AUDIO-STREAMING
2,10.152.152.11-216.58.220.99-57160-443-6,10.152.152.11,57160,216.58.220.99,443,6,24/07/2015 04:09:48 PM,431,1,1,...,0,0,0,0,0.000000e+00,0.000,0.000000e+00,0.000000e+00,Non-Tor,AUDIO-STREAMING
3,10.152.152.11-74.125.136.120-49134-443-6,10.152.152.11,49134,74.125.136.120,443,6,24/07/2015 04:09:48 PM,359,1,1,...,0,0,0,0,0.000000e+00,0.000,0.000000e+00,0.000000e+00,Non-Tor,AUDIO-STREAMING
4,10.152.152.11-173.194.65.127-34697-19305-6,10.152.152.11,34697,173.194.65.127,19305,6,24/07/2015 04:09:45 PM,10778451,591,400,...,0,0,0,0,1.437760e+15,3117718.131,1.437760e+15,1.437760e+15,Non-Tor,AUDIO-STREAMING


In [3]:
traffic_df["Label.1"].loc[traffic_df["Label.1"] == "AUDIO-STREAMING"] = "Audio-Streaming"
traffic_df["Label.1"].loc[traffic_df["Label.1"] == "File-transfer"] = "File-Transfer"
traffic_df["Label.1"].loc[traffic_df["Label.1"] == "Video-streaming"] = "Video-Streaming"

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [4]:
traffic_df.groupby('Label')['Label.1'].value_counts()

Label    Label.1        
Non-Tor  P2P                48300
         Browsing           32545
         File-Transfer       6740
         Video-Streaming     3382
         Audio-Streaming     1484
         Email                492
         Chat                 413
NonVPN   Chat                6524
         Email               5071
         Video-Streaming     5039
         Audio-Streaming     3296
         VOIP                2101
         File-Transfer       1832
Tor      VOIP                 298
         Browsing             263
         Audio-Streaming      224
         P2P                  220
         Video-Streaming      202
         File-Transfer        107
         Chat                  65
         Email                 13
VPN      Audio-Streaming    13060
         Chat                4476
         File-Transfer       2503
         VOIP                1167
         Video-Streaming     1144
         Email                569
Name: Label.1, dtype: int64

### Non-Tor, Non-Vpn

In [5]:
non_tor = traffic_df['Label'] == 'Non-Tor'
non_vpn = traffic_df['Label'] == 'NonVPN'

label_benign = non_tor | non_vpn
benign_df = traffic_df[label_benign]
benign_df['Label.1'].value_counts()

P2P                48300
Browsing           32545
File-Transfer       8572
Video-Streaming     8421
Chat                6937
Email               5563
Audio-Streaming     4780
VOIP                2101
Name: Label.1, dtype: int64

In [6]:
benign_df["Label.1"].loc[benign_df["Label.1"] == "Audio-Streaming"] = "benign_Audio-Streaming"
benign_df["Label.1"].loc[benign_df["Label.1"] == "Chat"] = "benign_Chat"
benign_df["Label.1"].loc[benign_df["Label.1"] == "File-Transfer"] = "benign_File-Transfer"
benign_df["Label.1"].loc[benign_df["Label.1"] == "VOIP"] = "benign_VOIP"
benign_df["Label.1"].loc[benign_df["Label.1"] == "Video-Streaming"] = "benign_Video-Streaming"
benign_df["Label.1"].loc[benign_df["Label.1"] == "Email"] = "benign_Email"
benign_df["Label.1"].loc[benign_df["Label.1"] == "Browsing"] = "benign_Browsing"
benign_df["Label.1"].loc[benign_df["Label.1"] == "P2P"] = "benign_P2P"

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [7]:
benign_df['Label.1'].value_counts()

benign_P2P                48300
benign_Browsing           32545
benign_File-Transfer       8572
benign_Video-Streaming     8421
benign_Chat                6937
benign_Email               5563
benign_Audio-Streaming     4780
benign_VOIP                2101
Name: Label.1, dtype: int64

### Tor, Vpn

In [8]:
tor = traffic_df['Label'] == 'Tor'
vpn = traffic_df['Label'] == 'VPN'

label_darknet = tor | vpn
darknet_df = traffic_df[label_darknet]
darknet_df['Label.1'].value_counts()

Audio-Streaming    13284
Chat                4541
File-Transfer       2610
VOIP                1465
Video-Streaming     1346
Email                582
Browsing             263
P2P                  220
Name: Label.1, dtype: int64

In [9]:
darknet_df["Label.1"].loc[darknet_df["Label.1"] == "Audio-Streaming"] = "dark_Audio-Streaming"
darknet_df["Label.1"].loc[darknet_df["Label.1"] == "Chat"] = "dark_Chat"
darknet_df["Label.1"].loc[darknet_df["Label.1"] == "File-Transfer"] = "dark_File-Transfer"
darknet_df["Label.1"].loc[darknet_df["Label.1"] == "VOIP"] = "dark_VOIP"
darknet_df["Label.1"].loc[darknet_df["Label.1"] == "Video-Streaming"] = "dark_Video-Streaming"
darknet_df["Label.1"].loc[darknet_df["Label.1"] == "Email"] = "dark_Email"
darknet_df["Label.1"].loc[darknet_df["Label.1"] == "Browsing"] = "dark_Browsing"
darknet_df["Label.1"].loc[darknet_df["Label.1"] == "P2P"] = "dark_P2P"

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [10]:
darknet_df['Label.1'].value_counts()

dark_Audio-Streaming    13284
dark_Chat                4541
dark_File-Transfer       2610
dark_VOIP                1465
dark_Video-Streaming     1346
dark_Email                582
dark_Browsing             263
dark_P2P                  220
Name: Label.1, dtype: int64

In [11]:
data = pd.concat([benign_df, darknet_df])
data['Label.1'].value_counts()

benign_P2P                48300
benign_Browsing           32545
dark_Audio-Streaming      13284
benign_File-Transfer       8572
benign_Video-Streaming     8421
benign_Chat                6937
benign_Email               5563
benign_Audio-Streaming     4780
dark_Chat                  4541
dark_File-Transfer         2610
benign_VOIP                2101
dark_VOIP                  1465
dark_Video-Streaming       1346
dark_Email                  582
dark_Browsing               263
dark_P2P                    220
Name: Label.1, dtype: int64

In [12]:
data.reset_index(drop=True, inplace=True)
data.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Label.1
0,10.152.152.11-216.58.220.99-57158-443-6,10.152.152.11,57158,216.58.220.99,443,6,24/07/2015 04:09:48 PM,229,1,1,...,0,0,0,0,0.000000e+00,0.000,0.000000e+00,0.000000e+00,Non-Tor,benign_Audio-Streaming
1,10.152.152.11-216.58.220.99-57159-443-6,10.152.152.11,57159,216.58.220.99,443,6,24/07/2015 04:09:48 PM,407,1,1,...,0,0,0,0,0.000000e+00,0.000,0.000000e+00,0.000000e+00,Non-Tor,benign_Audio-Streaming
2,10.152.152.11-216.58.220.99-57160-443-6,10.152.152.11,57160,216.58.220.99,443,6,24/07/2015 04:09:48 PM,431,1,1,...,0,0,0,0,0.000000e+00,0.000,0.000000e+00,0.000000e+00,Non-Tor,benign_Audio-Streaming
3,10.152.152.11-74.125.136.120-49134-443-6,10.152.152.11,49134,74.125.136.120,443,6,24/07/2015 04:09:48 PM,359,1,1,...,0,0,0,0,0.000000e+00,0.000,0.000000e+00,0.000000e+00,Non-Tor,benign_Audio-Streaming
4,10.152.152.11-173.194.65.127-34697-19305-6,10.152.152.11,34697,173.194.65.127,19305,6,24/07/2015 04:09:45 PM,10778451,591,400,...,0,0,0,0,1.437760e+15,3117718.131,1.437760e+15,1.437760e+15,Non-Tor,benign_Audio-Streaming


In [13]:
data.to_csv('/content/gdrive/MyDrive/CIC-Darknet/darknet.csv', index=False)